In [2]:
import copy
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../../../")
from utils.mujoco_parser import MuJoCoParserClass
from utils.util import sample_xyzs,rpy2r,r2quat
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


In [3]:
xml_path = '../../../asset/scene_realworld_w_shelf_category.xml'
env = MuJoCoParserClass(name='Place task scene: Office table',rel_xml_path=xml_path,VERBOSE=False, MODE='window')
print(env.MODE)

# Move tables and robot base
env.model.body('base_table').pos = np.array([0,0,0])
env.model.body('avoiding_object_table').pos = np.array([0.38+0.45,0,0])
env.model.body('base').pos = np.array([-0.18,0,0.79])
env.model.body('right_object_table').pos = np.array([-0.05,-0.80,0])
env.model.body('left_object_table').pos = np.array([-0.05,0.80,0])

# Place objects
tray_pos = np.array([ 0.9, 0.0, 0.8])
bowl_red_pos = np.array([ 0.9, 0.3, 0.8])
bowl_blue_pos = np.array([ 0.9, -0.3, 0.8])
env.model.joint(env.model.body('tray_gray').jntadr[0]).qpos0[:3] = tray_pos
env.model.joint(env.model.body('kitchen-bowl-red').jntadr[0]).qpos0[:3] = bowl_red_pos
env.model.joint(env.model.body('kitchen-bowl-blue').jntadr[0]).qpos0[:3] = bowl_blue_pos

env.model.joint(env.model.body('ycb-apple-2').jntadr[0]).qpos0[:3] = bowl_red_pos + np.array([0.0,0.0,0.02])
env.model.joint(env.model.body('ycb-banana-2').jntadr[0]).qpos0[:3] = tray_pos + np.array([0.0,0.03,0.2])
env.model.joint(env.model.body('ycb-banana-2').jntadr[0]).qpos0[3:] = r2quat(rpy2r(np.radians([0, 0, 90])))
env.model.joint(env.model.body('ycb-lemon-2').jntadr[0]).qpos0[:3] = tray_pos + np.array([-0.03,-0.07,0.02])
env.model.joint(env.model.body('ycb-orange-2').jntadr[0]).qpos0[:3] = bowl_blue_pos + np.array([0.0,0.0,0.02])

# Target objects
env.model.joint(env.model.body('ycb-apple').jntadr[0]).qpos0[:3] = np.array([0.1, -0.6, 0.8])
env.model.joint(env.model.body('ycb-banana').jntadr[0]).qpos0[:3] = np.array([-5.0,-1.5,0.2])
env.model.joint(env.model.body('ycb-lemon').jntadr[0]).qpos0[:3] = np.array([0.0, -0.6, 0.8])
env.model.joint(env.model.body('ycb-orange').jntadr[0]).qpos0[:3] = np.array([-0.1, -0.6, 0.8])

joint_names = env.rev_joint_names[:6]
idxs_forward = [env.model.joint(joint_name).qposadr[0] for joint_name in env.joint_names[:6]]
idxs_jacobian = [env.model.joint(joint_name).dofadr[0] for joint_name in env.joint_names[:6]]
list1, list2 = env.ctrl_joint_idxs, idxs_forward
idxs_step = []
for i in range(len(list2)):
    if list2[i] in list1:
        idxs_step.append(list1.index(list2[i]))

window


KeyError: "Invalid name 'tray_gray'. Valid names: ['avoiding_object_table', 'base', 'base_table', 'camera_center', 'camera_mount', 'chocopie', 'cider', 'coke', 'd435i', 'fanta_grape', 'forearm_link', 'kitchen-bowl-blue', 'kitchen-bowl-red', 'left_object_table', 'pocky', 'pringles', 'redbullcan', 'rg2_gripper_base_link', 'rg2_gripper_finger1_finger_link', 'rg2_gripper_finger1_finger_tip_link', 'rg2_gripper_finger1_inner_knuckle_link', 'rg2_gripper_finger2_finger_link', 'rg2_gripper_finger2_finger_tip_link', 'rg2_gripper_finger2_inner_knuckle_link', 'right_object_table', 'shelf_back', 'shelf_column1', 'shelf_column2', 'shelf_row1', 'shelf_row2', 'shelf_row3', 'shoulder_link', 'tcp_link', 'tray', 'upper_arm_link', 'world', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link', 'ycb-apple', 'ycb-apple-2', 'ycb-banana', 'ycb-banana-2', 'ycb-lemon', 'ycb-lemon-2', 'ycb-orange', 'ycb-orange-2']"